In [1]:
import pandas as pd

df=pd.read_csv("C:/Users/user/Downloads/injury.csv")
df['참조조문']=df['참조조문'].fillna('참조조문 없음')
df['참조조문'] = df['참조조문'].replace({'\n': '참조조문 없음'})
df['참조판례'] = df['참조판례'].replace({'\n': '참조판례 없음'})

In [2]:
import re

def clean_clause(text): # 전부 , 단위로 분리될수 있게 변환

    groups=text.replace('<br/>','').replace('[증명책임]','').replace('/',',').split(',')
    groups=[group for group in groups if group.strip() != '']
    results = []

    for group in groups:
        group = group.strip()  # 각 그룹의 양쪽 공백 제거
        group=re.sub(r'(가|나|다|라|마|바|사|아|자|차|카|타|파|하)\.', '', group) # 가., 나., 다., 라., 마., 바., 사., 아., 자., 차., 카., 타., 파., 하. 댜. 형태만 제거
        group=re.sub(r'(갸|냐|댜|랴|먀|뱌|샤|야|쟈|챠|캬|탸|퍄|햐)\.', '', group) # 가나다라의 오타도 제거
        
        group=re.sub(r'[\[\［]\d+[\]\］]\s*', '', group)  # '[1]', '[2]' 형식 제거
        group = group.strip()  # 각 그룹의 양쪽 공백 제거
        
        # 쉼표로 나눠 조각을 분리
        parts = re.split(r',\s*', group)                

        transformed_parts = []

        for part in parts:    # 조, 항, 호 뒤에 문자가 붙어 있는 경우 처리

            # 조, 항, 호 뒤에 괄호와 공백 없는 문자가 있는 경우 공백 추가 (괄호 예외 처리) 
            part = re.sub(r'([0-9]+조|[0-9]+항|[0-9]+호)(?![\s\(\[])', r'\1 ', part)          
            
            part = part.strip()
            # 변환된 조문 추가
            transformed_parts.append(part)

        # 각 그룹을 쉼표로 연결하고 결과에 추가
        if transformed_parts:  # transformed_parts가 비어있지 않으면
            results.append(', '.join(transformed_parts))

    # 전체 결과를 반환
    return ', '.join(results) 

ccl=df['참조조문'].apply(clean_clause) # 함수 적용
# 공백으로 이루어진 곳 제거하기 위해 ', ,'를 ','로 치환

In [20]:
def extract_law_names(series):
    law_names_set=set()
    pattern = r'([가-힣]+?(부칙|법\s*시행령|시행령|에\s*관한\s*법률|시행규칙|규칙|협정|의사록|관례|고시|대통령령|경제명령|법|령)(\([^)]*\))?)'
    # 정규 표현식을 사용하여 괄호를 기준으로 분리
    
    def split_outside_parentheses(text):
        result = []
        start = 0
        stack = []  # 괄호 스택

        for i, char in enumerate(text):
            if char in "{(":
                stack.append(char)  # 괄호 열기
            elif char in "})" and stack:
                stack.pop()  # 괄호 닫기
            elif char == "," and not stack:
                # 스택이 비어 있을 때의 쉼표는 분리 기준
                result.append(text[start:i].strip())
                start = i + 1

        # 마지막 남은 부분 추가
        result.append(text[start:].strip())
        return result

    for text in series:
        # 1. 쉼표로 문자열을 분리 (괄호 안 쉼표 무시)
        parts = split_outside_parentheses(text)
        for part in parts:
            # 불필요한 \n 및 공백 제거
            part = re.sub(r'\n+', ' ', part)
            part = re.sub(r'\s+', ' ', part)  # 여러 공백을 하나로 줄임
            part = re.sub(r'[\[\]]', '', part)  # [] 괄호 제거
            part = re.sub(r'〔\d+〕', '', part)
            part = part.strip()
            part = re.sub(r'\{[^}]*\}', '', part)  # 중괄호 안의 내용 제거

            split_pattern = r'([^()]*)(\s*\([^()]*\))?'

            matches = re.findall(split_pattern, part)
            matches = [(off_maren.replace(',', '').strip(), maren.strip()) for off_maren, maren in matches]

            for match in matches:
                before_parentheses = match[0].strip()  # 괄호 이전 내용
                in_parentheses = match[1].strip() if match[1] else ''  # 괄호 내용
                # '숫자. ' 형식이 있다면 제거
                before_parentheses = re.sub(r'^\d+\.\s*', '', before_parentheses)
                # print('in: ',in_parentheses)

                # 법 이름 추출: 중괄호가 제거된 `before_parentheses`에서 패턴 찾기
                match = re.search(pattern, before_parentheses)
                if match:
                    # 패턴이 일치하는 위치까지 추출
                    law_name = before_parentheses[:match.end()] + in_parentheses
                    if len(law_name) > 1:  # 길이가 1보다 큰 경우만 추가
                        law_names_set.add(law_name)

    return list(law_names_set)


# 결과 출력
law_names = extract_law_names(ccl)
print('추출된 법 이름:', law_names)

추출된 법 이름: ['영유아보육법', '행정소송법(입증책임)', '구 독점규제 및 공정거래에 관한 법률(2005. 3. 31. 대통령령 제18768호 로 개정되기 전의 것)', '제5조 제3항 민법', '근로기준법시행령', '가족관계의 등록 등에 관한 법률', '국제사법', '군인사법', '실화책임에 관한 법률', '구 폭력행위 등 처벌에 관한 법률(2016. 1. 6. 법률 제13718호 로 개정되기 전의 것)', '섭외사법', '노인복지법', '소송촉진 등에 관한 특례법', '여객자동차 운수사업법 시행령', '구 국회에서의 증언·감정 등에 관한 법률(2010. 3. 12. 법률 제10051호 로 개정되기 전의 것)', '육운진흥법시행령', '약사법', '구 직업안정및고용촉진에관한법률(1994. 1. 7. 법률 제4733호 직업안정법으로 전문 개정되기 전의 것)', '국적법', '자동차손해배상 보장법 시행령', '제2조 제1항 헌법', '민법(제396조 )', '구 자동차손해배상 보장법 시행령(1999. 6. 30. 대통령령 제16463호 로 전문 개정되기 전의 것)', '구 동물보호법(2022. 4. 26. 법률 제18853호 로 전부 개정되기 전의 것)', '형법', '제조물 책임법', '구 도로교통법(2016. 12. 2. 법률 제14356호 로 개정되기 전의 것)', '공동주택관리령', '노동조합 및 노동관계조정법 시행령', '소송촉진등에 관한 특례법', '국립묘지의 설치 및 운영에 관한 법률', '구 자동차손해배상 보장법 시행령(2021. 1. 5. 대통령령 제31380호 로 개정되기 전의 것)', '의료법 시행령', '경찰관직무집행법 시행령', '구 성폭력범죄의 처벌 등에 관한 특례법(2016. 12. 20. 법률 제14412호 로 개정되기 전의 것)', '구 의료법(2006. 10. 27. 법률 제8067호 로 개정되기 전의 것)', '도로교통법(가)', '구 경찰공무원법(1996. 8. 8. 법률 제5153호 로 개정되기 전의 것)', '구국가

In [22]:
def clean_extracted_law_names(law_names):
    cleaned_law_names = []
    # 패턴 뒤의 괄호와 '구 ' 접두사 제거
    for law in law_names:
        # "제3조", "제4항", "제5호", "제6목" 등과 같은 패턴 제거
        law = re.sub(r'제\d+(조|항|호|목)\s*', '', law)
        
        # '구 ' 접두사 제거
        law = law.replace('구 ', '')
        
        # 패턴 뒤의 괄호 제거
        # 예: "폭력행위 등 처벌에 관한 법률(2014. 12. 30. 법률 제12896호로 개정되기 전의 것)" -> "폭력행위 등 처벌에 관한 법률"
        law = re.sub(r'(부칙|법\s*시행령|시행령|에\s*관한\s*법률|시행규칙|규칙|협정|의사록|관례|고시|대통령령|경제명령|법|령)(\([^)]*\))', r'\1', law)
        
        # 앞쪽의 마침표와 공백 제거
        law = re.sub(r'^\s*\.?\s*', '', law)        

        # '\\ ' 제거
        law = re.sub(r'\\\s*', '', law)
                
        cleaned_law_names.append(law.strip())  # 최종적으로 공백 제거 후 리스트에 추가
    
    return cleaned_law_names

In [23]:
law_names_cleaned=clean_extracted_law_names(law_names)
law_dup_reduce=list(set(law_names_cleaned))
# 법 이름 길이가 긴 순서대로 정렬 (내림차순)
law_names = sorted(law_names, key=len, reverse=True)

In [ ]:
law_names.append("구 주택법(2016. 1. 19. 법률 제13805호 로 전부 개정되기 전의 것)")
law_names.append("주택법(2016. 1. 19. 법률 제13805호 로 전부 개정되기 전의 것)")

In [41]:
law_names.append("구 주택공급에 관한 규칙(2015. 9. 1. 국토교통부령 제227호 로 개정되기 전의 것)")

In [42]:
import re

def process_law_references(a):
    if isinstance(a, str):  
        previous_law = None
        new_references = []
        
        # 쉼표를 기준으로 나누되, 괄호 안의 쉼표는 무시
        parts = re.split(r',(?![^()]*\))', a)
        
        for part in parts:
            print('1.회차시작: ',part)
            part = part.strip()
            part = re.sub(r'[\[\]［］].*?[\]\］]', '', part)
            part = re.sub(r'\s*[가-힣]\.\s*', '', part)
            part = part.strip('<br/>')
            part = part.replace('/', ',')
            part = re.sub(r'\s+', ' ', part).strip()
            
            
            
            # 현재 조각에 법 이름이 있는 경우, previous_law 업데이트
            for law_name in law_names:
                if law_name in part:
                   law_name = law_name.replace('구 ', '')
                   print('여기 들ㅇ거ㅏㅁ')
                   # 패턴 뒤의 괄호 제거
                   law_name = re.sub(r'(부칙|법\s*시행령|시행령|에\s*관한\s*법률|시행규칙|규칙|협정|의사록|관례|고시|대통령령|경제명령|법|령)(\([^)]*\))', r'\1', law_name)   

                   previous_law = law_name  # 이전 법 이름을 현재 법 이름으로 설정

                   part = re.sub(r'구\s*', '', part)  # 구문에서 '구 ' 제거

                   part = re.sub(r'(부칙|법\s*시행령|시행령|에\s*관한\s*법률|시행규칙|규칙|협정|의사록|관례|고시|대통령령|경제명령|법|령)(\([^)]*\))', r'\1', part)

                   break
                      
                      
                      
            print('0. 이전법 : ',previous_law)
            # 1. 괄호 안에 "(현행 ~ 참조)" 패턴이 있는지 탐색
            match = re.search(r'\(현행 (.*?) 참조\)', part)
            print('2.현행~ 매치확인 :',match)
            print('ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ')
            if match is None:
                # 1-1. "(현행 ~ 참조)"가 없고 previous_law 존재하고 part에 없는 경우, 법 이름을 붙인다.
                if previous_law and re.match(r'제\d+조', part) and not any(law in part for law in law_dup_reduce) and previous_law not in part:
                    part = f"{previous_law} {part}"
                    print("2-1.현행매치가 없는경우 파트: ",part)
                else:
                    print('매치도 없고 현행법도 없고 파트에 법도없다')
                    
            else:
                # "(현행 ~ 참조)"가 존재하는 경우
                content_between = match.group(1)
                print('2-2.안내용확인 ',content_between)
                # 2. "(현행 ~ 참조)" 구문 안에 법 이름 패턴이 있는지 확인
                in_pattern = r'([가-힣]+?(법\s*시행령|시행령|에\s*관한\s*법률|시행규칙|규칙|협정|의사록|관례|고시|대통령령|경제명령|법|령))'
                law_match = re.search(in_pattern, content_between)
                print(law_match)
                if law_match is None: # (현행 제34조 참조)  이런거밖에 없다는 뜻
                    print('2-2-1.현행참조 법이 없습니다.')
                    # 2-1. 법 이름이 없는 경우, 괄호 밖의 법 이름을 사용하여 내용을 변환
                    # currnet_law=previous_law
                    if previous_law:
                        if re.search(r'제\d+조', content_between):
                            part = f"{previous_law} {content_between}"
                            print('2-1.법 이름 없는경우 맞춤',part)
                    else:
                        part = content_between
                        print('2-1-1. 법이름이 없는데 조도 없을때', part)
                else:
                    # 2-2. 법 이름이 존재하는 경우, current_law를 설정하고 ','로 나눈 후 처리
                    current_law = law_match.group(0).strip()
                    print('2-2-2.현재법',current_law)
                    # ','로 구분된 각 조항 처리
                    references = content_between.split(',')
                    print('2-2-2 에서 분리 된 것들 :',references)
                    updated_references = []
                    for ref in references:
                        ref = ref.strip()
                        
                        # 2-2-1. '제%d조' 형식이 있다면 current_law를 붙이기
                        if re.match(r'제\d+조', ref):
                            ref = f"{current_law} {ref}"
                        
                        # 2-2-1. ',' 뒤에 또 다른 법 이름이 있으면 current_law 갱신
                        next_law_match = re.search(in_pattern, ref)
                        if next_law_match:
                            current_law = next_law_match.group(0).strip()
                        updated_references.append(ref)
                        print('분리된것 변경',updated_references)

                    # 2-3. 변환된 내용으로 part 전체를 대체
                    updated_content = ', '.join(updated_references)
                    print('2-3.분리한거 합치기',updated_content)
                    part = updated_content
            
            # 처리된 part를 new_references에 추가
            new_references.append(part)
            print('변경된 파트: ',part)
            print('ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ')
            print('ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ')
        # 최종 조각들을 쉼표로 결합하여 반환
        return ', '.join(new_references)

# 예제 호출
a = "구 독점규제 및 공정거래에 관한 법률(2005. 3. 31. 대통령령 제18768호 로 개정되기 전의 것) 제75조(현행 바보법 제56조, 제68조 참조), 구 주택법(2016. 1. 19. 법률 제13805호 로 전부 개정되기 전의 것) 부칙(2015. 6. 22.) 제5조, 구 주택공급에 관한 규칙(2015. 9. 1. 국토교통부령 제227호 로 개정되기 전의 것) 제5조 제3항(현행 주택법 제56조 제4항 참조), 제5조 의2 제1항(현행 삭제), 제2항(현행 삭제), 제6항(현행 삭제), 제5조 의5 제1항 제1호(현행 제12조 제1항 참조), 민법 제264조, 제278조"
# law_names = ["주택법", "주택공급에 관한 규칙", "바보법", "민법"]
result = process_law_references(a)
print("최종 결과:", result)

1.회차시작:  구 독점규제 및 공정거래에 관한 법률(2005. 3. 31. 대통령령 제18768호 로 개정되기 전의 것) 제75조(현행 바보법 제56조, 제68조 참조)
여기 들ㅇ거ㅏㅁ
0. 이전법 :  독점규제 및 공정거래에 관한 법률
2.현행~ 매치확인 : <re.Match object; span=(23, 45), match='(현행 바보법 제56조, 제68조 참조)'>
ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
2-2.안내용확인  바보법 제56조, 제68조
<re.Match object; span=(0, 3), match='바보법'>
2-2-2.현재법 바보법
2-2-2 에서 분리 된 것들 : ['바보법 제56조', ' 제68조']
분리된것 변경 ['바보법 제56조']
분리된것 변경 ['바보법 제56조', '바보법 제68조']
2-3.분리한거 합치기 바보법 제56조, 바보법 제68조
변경된 파트:  바보법 제56조, 바보법 제68조
ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
1.회차시작:   구 주택법(2016. 1. 19. 법률 제13805호 로 전부 개정되기 전의 것) 부칙(2015. 6. 22.) 제5조
여기 들ㅇ거ㅏㅁ
0. 이전법 :  주택법
2.현행~ 매치확인 : None
ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
매치도 없고 현행법도 없고 파트에 법도없다
변경된 파트:  주택법 부칙 제5조
ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
1.회차시작:   구 주택공급에 관한 규칙(2015. 9. 1. 국토교통부령 제227호 로 개정되기 전의 것) 제5조 제3항(현행 주택법 제56조 제4항 참조)
여기 들ㅇ거ㅏㅁ
0. 이전법 :  주택공급

# ㄴ>위에가 일단 체크

문제지점: match is None 일시 previous_law를 불러와서 앞에 붙이는게 없어짐. 특히 '제~조~'하는 곳에서

문제지점: match True 일시 not law_match 일 때 변경된 파트에서 previous_law에다가 괄호안 내용 하는게 없음. 그냥 part그대로 가져옴.

문제지점: 부칙 이 있으면 앞에있는 법까지만 짤라서 previous_law에 넣어지고 나머지는 그대로.

문제지점: previous_law가 제대로 업데이트가 안됨. """구 주택공급에 관한 규칙(2015. 9. 1. 국토교통부령 제227호 로 개정되기 전의 것)""" 이걸 """독점규제 및 공정거래에 관한 법률"""이렇게 이전 previous_law를 가져옴

문제지점: previous_law가 업데이트가 안됨; (구 주택공급에 관한 규칙(2015. 9. 1. 국토교통부령 제227호 로 개정되기 전의 것) 을 추출했는데 previous_law가 주택법 그대로임) 자료가 잘못되어 law_names에 없어서 그런거 같음.



In [39]:
def process_law_references(a):
    if isinstance(a, str):  
        previous_law = None
        new_references = []
        
        # 쉼표를 기준으로 나누되, 괄호 안의 쉼표는 무시
        parts = re.split(r',(?![^()]*\))', a)
        
        for part in parts:
            part = part.strip()
            part = re.sub(r'[\[\]［］].*?[\]\］]', '', part)
            part = re.sub(r'\s*[가-힣]\.\s*', '', part)
            part = part.replace('/', ',')
            part = re.sub(r'\s+', ' ', part).strip()
            
            # 현재 조각에 법 이름이 있는 경우, previous_law 업데이트
            for law_name in law_names:
                if law_name in part:
                    law_name = law_name.replace('구 ', '')
                    law_name = re.sub(r'(부칙|법\s*시행령|시행령|에\s*관한\s*법률|시행규칙|규칙|협정|의사록|관례|고시|대통령령|경제명령|법|령)(\([^)]*\))', r'\1', law_name)
                    previous_law = law_name
                    part = re.sub(r'구\s*', '', part)
                    part = re.sub(r'(부칙|법\s*시행령|시행령|에\s*관한\s*법률|시행규칙|규칙|협정|의사록|관례|고시|대통령령|경제명령|법|령)(\([^)]*\))', r'\1', part)
                    break
            
            # 1. 괄호 안에 "(현행 ~ 참조)" 패턴이 있는지 탐색
            match = re.search(r'\(현행 (.*?) 참조\)', part)
            if match:
                # "(현행 ~ 참조)"가 존재하는 경우
                content_between = match.group(1)
                in_pattern = r'([가-힣]+?(법\s*시행령|시행령|에\s*관한\s*법률|시행규칙|규칙|협정|의사록|관례|고시|대통령령|경제명령|법|령))'
                law_match = re.search(in_pattern, content_between)
                
                if law_match:
                    # 현행 참조에 법 이름이 있을 때, 해당 법 이름 사용
                    current_law = law_match.group(0).strip()
                else:
                    # 법 이름이 없으면 previous_law를 사용
                    current_law = previous_law

                # ','로 구분된 각 조항 처리
                references = content_between.split(',')
                updated_references = []
                for ref in references:
                    ref = ref.strip()
                    # '제%d조' 형식일 때만 current_law를 붙임
                    if re.match(r'제\d+조', ref):
                        ref = f"{current_law} {ref}"
                    updated_references.append(ref)
                    
                # 변환된 내용으로 part 대체
                part = ', '.join(updated_references)
            elif previous_law and re.match(r'제\d+조', part) and previous_law not in part:
                # '제%d조' 형식이 있을 때에만 previous_law를 붙임
                part = f"{previous_law} {part}"
            
            # 처리된 part를 new_references에 추가
            new_references.append(part)
        
        # 최종 조각들을 쉼표로 결합하여 반환
        return ', '.join(new_references)

# 예제 호출
a = "구 독점규제 및 공정거래에 관한 법률(2005. 3. 31. 대통령령 제18768호 로 개정되기 전의 것) 제75조(현행 바보법 제56조, 제68조 참조), 구 주택법(2016. 1. 19. 법률 제13805호 로 전부 개정되기 전의 것) 부칙(2015. 6. 22.) 제5조, 구 주택공급에 관한 규칙(2015. 9. 1. 국토교통부령 제227호 로 개정되기 전의 것) 제5조 제3항(현행 주택법 제56조 제4항 참조), 제5조 의2 제1항(현행 삭제), 제2항(현행 삭제), 제6항(현행 삭제), 제5조 의5 제1항 제1호(현행 제12조 제1항 참조), 민법 제264조, 제278조"
result = process_law_references(a)
print("최종 결과:", result)

최종 결과: 바보법 제56조, 바보법 제68조, 주택법 부칙 제5조, 주택법 제56조 제4항, 주택법 제5조 의2 제1항(현행 삭제), 제2항(현행 삭제), 제6항(현행 삭제), 주택법 제12조 제1항, 민법 제264조, 민법 제278조
